In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import torch
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import torch
import faiss
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

import os
import sys
import inspect

sys.path.insert(1, os.path.abspath( os.path.dirname('../')) )

from dataloader.anaxnet import CustomDataset
from model.anaxnet_attn_multilayer import CustomModel
from common_metrics import compute_metrics

In [2]:
nine_class_labels = [
    'lung opacity', 
    'pleural effusion', 
    'atelectasis', 
    'enlarged cardiac silhouette',
    'pulmonary edema/hazy opacity', 
    'pneumothorax', 
    'consolidation', 
    'fluid overload/heart failure', 
    'pneumonia']

anatomy_names = [
    'right lung',
    'right apical zone',
    'right upper lung zone',
    'right mid lung zone',
    'right lower lung zone',
    'right hilar structures',
    'right costophrenic angle',
    'left lung',
    'left apical zone',
    'left upper lung zone',
    'left mid lung zone',
    'left lower lung zone',
    'left hilar structures',
    'left costophrenic angle',
    'mediastinum',
    'upper mediastinum',
    'cardiac silhouette',
    'trachea']

# Top-5

In [3]:
df_gt = pd.read_csv('../results/graph_transformer/best_config_abs_pos_with_edges/occluded/orig/9_classes_auc_no_global_feat_top_5_cosine_.txt')
df_anax = pd.read_csv('../results/anaxnet/anaxnet/occluded/orig/9_classes_auc_no_global_feat_top_5_cosine_.txt')
df_ath = pd.read_csv('../results/ath/ath/orig/9_classes_auc_no_global_feat_top_5_cosine.txt')
df_res = pd.read_csv('../results/resnet50/resnet50_fc_new/orig/9_classes_auc_no_global_feat_top_5_cosine.txt')

In [46]:
t = PrettyTable()
t.field_names = ["Disease", "Query Count"]
t.title = "Query Count"
weights = []

for class_name in nine_class_labels:
    t.add_row([class_name, len(df_gt[df_gt['label_name'] == class_name])])
    weights.append(len(df_gt[df_gt['label_name'] == class_name]))

print(t)

+--------------------------------------------+
|                Query Count                 |
+------------------------------+-------------+
|           Disease            | Query Count |
+------------------------------+-------------+
|         lung opacity         |     2679    |
|       pleural effusion       |     1340    |
|         atelectasis          |     1241    |
| enlarged cardiac silhouette  |     1212    |
| pulmonary edema/hazy opacity |     819     |
|         pneumothorax         |      96     |
|        consolidation         |     334     |
| fluid overload/heart failure |     169     |
|          pneumonia           |     643     |
+------------------------------+-------------+


## P-Value

In [5]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & AnaxNet'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt = df_gt[(df_gt['label_name'] == class_label)][metric].to_numpy()
        anax_mt = df_anax[(df_anax['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_anax = ttest_rel(gt_mt, anax_mt)

        entry.append(np.round(ttest_gt_anax.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                     GT & AnaxNet                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  | 0.004 |
|       pleural effusion       |  0.0  |  0.0  |  0.0  |
|         atelectasis          |  0.0  |  0.0  | 0.003 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         | 0.919 | 0.665 | 0.896 |
|        consolidation         | 0.329 | 0.168 | 0.304 |
| fluid overload/heart failure | 0.001 |  0.0  | 0.001 |
|          pneumonia           | 0.013 | 0.008 | 0.005 |
+------------------------------+-------+-------+-------+


In [6]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ATH'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt = df_gt[(df_gt['label_name'] == class_label)][metric].to_numpy()
        ath_mt = df_ath[(df_ath['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_ath = ttest_rel(gt_mt, ath_mt)

        entry.append(np.round(ttest_gt_ath.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                       GT & ATH                       |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  |  0.0  |
|       pleural effusion       |  0.0  |  0.0  |  0.0  |
|         atelectasis          |  0.0  |  0.0  |  0.0  |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         |  0.0  |  0.0  |  0.0  |
|        consolidation         | 0.011 | 0.014 | 0.007 |
| fluid overload/heart failure |  0.0  |  0.0  |  0.0  |
|          pneumonia           |  0.0  |  0.0  |  0.0  |
+------------------------------+-------+-------+-------+


In [7]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ResNet50'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt = df_gt[(df_gt['label_name'] == class_label)][metric].to_numpy()
        res_mt = df_res[(df_res['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_res = ttest_rel(gt_mt, res_mt)

        entry.append(np.round(ttest_gt_res.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                    GT & ResNet50                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  | 0.038 |
|       pleural effusion       | 0.009 |  0.0  | 0.082 |
|         atelectasis          | 0.001 |  0.0  | 0.005 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         | 0.025 | 0.001 | 0.021 |
|        consolidation         | 0.032 |  0.22 | 0.039 |
| fluid overload/heart failure |  0.0  | 0.001 |  0.0  |
|          pneumonia           |  0.0  |  0.0  |  0.0  |
+------------------------------+-------+-------+-------+


# Top-3

In [11]:
df_gt_3 = pd.read_csv('../results/graph_transformer/best_config_abs_pos_with_edges/occluded/orig/9_classes_auc_no_global_feat_top_3_cosine_.txt')
df_anax_3 = pd.read_csv('../results/anaxnet/anaxnet/occluded/orig/9_classes_auc_no_global_feat_top_3_cosine_.txt')
df_ath_3 = pd.read_csv('../results/ath/ath/orig/9_classes_auc_no_global_feat_top_3_cosine.txt')
df_res_3 = pd.read_csv('../results/resnet50/resnet50_fc_new/orig/9_classes_auc_no_global_feat_top_3_cosine.txt')

In [47]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'GT'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_gt_3[(df_gt_3['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_gt_3[(df_gt_3['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_gt_3[(df_gt_3['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                          GT                          |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.925 | 0.882 |  0.93 |
|       pleural effusion       | 0.721 | 0.594 |  0.73 |
|         atelectasis          | 0.649 | 0.485 | 0.658 |
| enlarged cardiac silhouette  | 0.705 | 0.554 | 0.717 |
| pulmonary edema/hazy opacity |  0.64 | 0.491 | 0.652 |
|         pneumothorax         | 0.297 | 0.149 | 0.297 |
|        consolidation         | 0.272 | 0.159 | 0.277 |
| fluid overload/heart failure | 0.263 | 0.136 | 0.264 |
|          pneumonia           | 0.448 | 0.282 | 0.454 |
|           Average            | 0.547 | 0.415 | 0.553 |
|       Weighted Average       | 0.713 | 0.598 |  0.72 |
+------------------------------+-------+-------+-------+


In [49]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'AnaxNet'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_anax_3[(df_anax_3['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_anax_3[(df_anax_3['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_anax_3[(df_anax_3['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                       AnaxNet                        |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.907 |  0.84 | 0.917 |
|       pleural effusion       | 0.648 | 0.486 | 0.657 |
|         atelectasis          | 0.597 | 0.425 | 0.611 |
| enlarged cardiac silhouette  | 0.645 | 0.483 | 0.652 |
| pulmonary edema/hazy opacity | 0.537 | 0.363 | 0.545 |
|         pneumothorax         |  0.32 | 0.163 | 0.321 |
|        consolidation         | 0.256 |  0.15 | 0.258 |
| fluid overload/heart failure |  0.15 | 0.077 | 0.151 |
|          pneumonia           | 0.394 | 0.262 | 0.397 |
|           Average            | 0.495 | 0.361 | 0.501 |
|       Weighted Average       | 0.663 | 0.534 | 0.671 |
+------------------------------+-------+-------+-------+


In [50]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'ATH'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_ath_3[(df_ath_3['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_ath_3[(df_ath_3['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_ath_3[(df_ath_3['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                         ATH                          |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.897 | 0.825 | 0.906 |
|       pleural effusion       | 0.626 | 0.476 | 0.639 |
|         atelectasis          | 0.539 | 0.368 | 0.545 |
| enlarged cardiac silhouette  | 0.616 | 0.459 | 0.625 |
| pulmonary edema/hazy opacity |  0.51 |  0.36 | 0.517 |
|         pneumothorax         | 0.076 | 0.042 | 0.078 |
|        consolidation         | 0.209 |  0.13 | 0.211 |
| fluid overload/heart failure | 0.136 | 0.073 | 0.137 |
|          pneumonia           | 0.343 | 0.214 | 0.346 |
|           Average            | 0.439 | 0.327 | 0.445 |
|       Weighted Average       | 0.632 |  0.51 |  0.64 |
+------------------------------+-------+-------+-------+


In [51]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'ResNet50'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_res_3[(df_res_3['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_res_3[(df_res_3['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_res_3[(df_res_3['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                       ResNet50                       |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.914 | 0.858 | 0.919 |
|       pleural effusion       |  0.7  | 0.562 | 0.712 |
|         atelectasis          | 0.608 | 0.452 | 0.619 |
| enlarged cardiac silhouette  | 0.645 | 0.493 | 0.656 |
| pulmonary edema/hazy opacity | 0.553 | 0.407 | 0.561 |
|         pneumothorax         | 0.207 | 0.115 |  0.21 |
|        consolidation         | 0.216 | 0.129 | 0.218 |
| fluid overload/heart failure | 0.112 | 0.065 | 0.113 |
|          pneumonia           | 0.371 | 0.228 | 0.375 |
|           Average            | 0.481 | 0.368 | 0.487 |
|       Weighted Average       | 0.671 | 0.557 | 0.679 |
+------------------------------+-------+-------+-------+


## P-Value

In [12]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & AnaxNet'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_3 = df_gt_3[(df_gt_3['label_name'] == class_label)][metric].to_numpy()
        anax_mt_3 = df_anax_3[(df_anax_3['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_anax_3 = ttest_rel(gt_mt_3, anax_mt_3)

        entry.append(np.round(ttest_gt_anax_3.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                     GT & AnaxNet                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  | 0.006 |
|       pleural effusion       |  0.0  |  0.0  |  0.0  |
|         atelectasis          |  0.0  |  0.0  | 0.001 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         |  0.61 | 0.602 | 0.599 |
|        consolidation         | 0.535 | 0.563 |  0.48 |
| fluid overload/heart failure | 0.002 | 0.002 | 0.002 |
|          pneumonia           | 0.008 | 0.132 | 0.007 |
+------------------------------+-------+-------+-------+


In [13]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ATH'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_3 = df_gt_3[(df_gt['label_name'] == class_label)][metric].to_numpy()
        ath_mt_3 = df_ath_3[(df_ath_3['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_ath_3 = ttest_rel(gt_mt_3, ath_mt_3)

        entry.append(np.round(ttest_gt_ath_3.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                       GT & ATH                       |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  |  0.0  |
|       pleural effusion       |  0.0  |  0.0  |  0.0  |
|         atelectasis          |  0.0  |  0.0  |  0.0  |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         |  0.0  |  0.0  |  0.0  |
|        consolidation         | 0.018 | 0.058 | 0.014 |
| fluid overload/heart failure |  0.0  |  0.0  |  0.0  |
|          pneumonia           |  0.0  |  0.0  |  0.0  |
+------------------------------+-------+-------+-------+


In [14]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ResNet50'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_3 = df_gt_3[(df_gt_3['label_name'] == class_label)][metric].to_numpy()
        res_mt_3 = df_res_3[(df_res_3['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_res_3 = ttest_rel(gt_mt_3, res_mt_3)

        entry.append(np.round(ttest_gt_res_3.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                    GT & ResNet50                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.013 |  0.0  | 0.029 |
|       pleural effusion       | 0.073 | 0.003 | 0.147 |
|         atelectasis          | 0.004 | 0.004 | 0.008 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         | 0.083 | 0.206 | 0.096 |
|        consolidation         | 0.039 | 0.059 | 0.033 |
| fluid overload/heart failure |  0.0  |  0.0  |  0.0  |
|          pneumonia           |  0.0  |  0.0  |  0.0  |
+------------------------------+-------+-------+-------+


# Top-10

In [9]:
df_gt_10 = pd.read_csv('../results/graph_transformer/best_config_abs_pos_with_edges/occluded/orig/9_classes_auc_no_global_feat_top_10_cosine_.txt')
df_anax_10 = pd.read_csv('../results/anaxnet/anaxnet/occluded/orig/9_classes_auc_no_global_feat_top_10_cosine_.txt')
df_ath_10 = pd.read_csv('../results/ath/ath/orig/9_classes_auc_no_global_feat_top_10_cosine.txt')
df_res_10 = pd.read_csv('../results/resnet50/resnet50_fc_new/orig/9_classes_auc_no_global_feat_top_10_cosine.txt')

In [52]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'GT'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_gt_10[(df_gt_10['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_gt_10[(df_gt_10['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_gt_10[(df_gt_10['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                          GT                          |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.905 | 0.877 | 0.932 |
|       pleural effusion       | 0.676 | 0.575 | 0.751 |
|         atelectasis          | 0.597 | 0.476 | 0.686 |
| enlarged cardiac silhouette  | 0.646 | 0.531 | 0.741 |
| pulmonary edema/hazy opacity | 0.592 | 0.463 |  0.68 |
|         pneumothorax         | 0.314 | 0.115 | 0.354 |
|        consolidation         | 0.307 | 0.141 | 0.338 |
| fluid overload/heart failure | 0.298 | 0.107 | 0.321 |
|          pneumonia           | 0.428 | 0.255 | 0.507 |
|           Average            | 0.529 | 0.393 |  0.59 |
|       Weighted Average       | 0.679 | 0.583 | 0.743 |
+------------------------------+-------+-------+-------+


In [53]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'AnaxNet'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_anax_10[(df_anax_10['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_anax_10[(df_anax_10['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_anax_10[(df_anax_10['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                       AnaxNet                        |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.873 | 0.835 | 0.919 |
|       pleural effusion       | 0.593 | 0.464 | 0.689 |
|         atelectasis          | 0.551 | 0.414 |  0.65 |
| enlarged cardiac silhouette  | 0.595 | 0.461 | 0.683 |
| pulmonary edema/hazy opacity | 0.502 |  0.34 | 0.591 |
|         pneumothorax         | 0.321 | 0.095 | 0.353 |
|        consolidation         | 0.272 | 0.113 | 0.307 |
| fluid overload/heart failure | 0.189 | 0.063 | 0.203 |
|          pneumonia           | 0.395 | 0.224 | 0.455 |
|           Average            | 0.477 | 0.334 | 0.539 |
|       Weighted Average       | 0.628 | 0.517 | 0.699 |
+------------------------------+-------+-------+-------+


In [54]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'ATH'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_ath_10[(df_ath_10['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_ath_10[(df_ath_10['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_ath_10[(df_ath_10['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                         ATH                          |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         | 0.862 |  0.82 | 0.909 |
|       pleural effusion       | 0.582 | 0.452 | 0.672 |
|         atelectasis          | 0.505 | 0.361 | 0.592 |
| enlarged cardiac silhouette  | 0.573 | 0.438 | 0.657 |
| pulmonary edema/hazy opacity | 0.484 | 0.328 | 0.561 |
|         pneumothorax         | 0.103 | 0.043 | 0.113 |
|        consolidation         | 0.248 | 0.108 | 0.262 |
| fluid overload/heart failure | 0.166 | 0.067 | 0.183 |
|          pneumonia           | 0.362 | 0.222 | 0.416 |
|           Average            | 0.432 | 0.315 | 0.485 |
|       Weighted Average       | 0.605 | 0.497 | 0.671 |
+------------------------------+-------+-------+-------+


In [55]:
t = PrettyTable()
t.field_names = ['Disease', 'mAP', 'mHR', 'mRR']
t.title = 'ResNet50'

ap, hr, rr = [], [], []
for class_label in nine_class_labels:
    mAP = np.mean(df_res_10[(df_res_10['label_name'] == class_label)]['AP'].to_numpy())
    mHR = np.mean(df_res_10[(df_res_10['label_name'] == class_label)]['HR'].to_numpy())
    mRR = np.mean(df_res_10[(df_res_10['label_name'] == class_label)]['RR'].to_numpy())
    t.add_row([class_label, np.round(mAP, 3), np.round(mHR, 3), np.round(mRR, 3)])
    ap.append(mAP)
    hr.append(mHR)
    rr.append(mRR)

t.add_row(['Average', np.round(np.mean(ap), 3), np.round(np.mean(hr), 3), np.round(np.mean(rr), 3)])
t.add_row(['Weighted Average', np.round(np.average(ap, weights=weights), 3), np.round(np.average(hr, weights=weights), 3), np.round(np.average(rr, weights=weights), 3)])

print(t)

+------------------------------------------------------+
|                       ResNet50                       |
+------------------------------+-------+-------+-------+
|           Disease            |  mAP  |  mHR  |  mRR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.89 | 0.861 | 0.922 |
|       pleural effusion       | 0.644 | 0.541 | 0.732 |
|         atelectasis          | 0.557 | 0.435 |  0.65 |
| enlarged cardiac silhouette  | 0.598 | 0.485 | 0.685 |
| pulmonary edema/hazy opacity | 0.523 | 0.391 | 0.602 |
|         pneumothorax         | 0.236 | 0.068 | 0.246 |
|        consolidation         | 0.263 | 0.132 | 0.289 |
| fluid overload/heart failure | 0.165 | 0.067 | 0.169 |
|          pneumonia           | 0.381 | 0.223 | 0.442 |
|           Average            | 0.473 | 0.356 | 0.526 |
|       Weighted Average       | 0.642 | 0.549 | 0.705 |
+------------------------------+-------+-------+-------+


## P-Value

In [15]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & AnaxNet'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_10 = df_gt_10[(df_gt_10['label_name'] == class_label)][metric].to_numpy()
        anax_mt_10 = df_anax_10[(df_anax_10['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_anax_10 = ttest_rel(gt_mt_10, anax_mt_10)

        entry.append(np.round(ttest_gt_anax_10.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                     GT & AnaxNet                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  | 0.004 |
|       pleural effusion       |  0.0  |  0.0  |  0.0  |
|         atelectasis          |  0.0  |  0.0  | 0.004 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         | 0.854 | 0.113 | 0.979 |
|        consolidation         | 0.103 | 0.001 | 0.203 |
| fluid overload/heart failure |  0.0  |  0.0  | 0.001 |
|          pneumonia           | 0.013 |  0.0  | 0.005 |
+------------------------------+-------+-------+-------+


In [16]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ATH'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_10 = df_gt_10[(df_gt['label_name'] == class_label)][metric].to_numpy()
        ath_mt_10 = df_ath_10[(df_ath_10['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_ath_10 = ttest_rel(gt_mt_10, ath_mt_10)

        entry.append(np.round(ttest_gt_ath_10.pvalue, 3))
    
    t.add_row(entry)

print(t)

+----------------------------------------------------+
|                      GT & ATH                      |
+------------------------------+-------+-----+-------+
|           Disease            |   AP  |  HR |   RR  |
+------------------------------+-------+-----+-------+
|         lung opacity         |  0.0  | 0.0 |  0.0  |
|       pleural effusion       |  0.0  | 0.0 |  0.0  |
|         atelectasis          |  0.0  | 0.0 |  0.0  |
| enlarged cardiac silhouette  |  0.0  | 0.0 |  0.0  |
| pulmonary edema/hazy opacity |  0.0  | 0.0 |  0.0  |
|         pneumothorax         |  0.0  | 0.0 |  0.0  |
|        consolidation         | 0.006 | 0.0 | 0.002 |
| fluid overload/heart failure |  0.0  | 0.0 |  0.0  |
|          pneumonia           |  0.0  | 0.0 |  0.0  |
+------------------------------+-------+-----+-------+


In [17]:
t = PrettyTable()
t.field_names = ['Disease', 'AP', 'HR', 'RR']
t.title = 'GT & ResNet50'

for class_label in nine_class_labels:

    entry = [class_label]
    for metric in ['AP', 'HR', 'RR']:
        gt_mt_10 = df_gt_10[(df_gt_10['label_name'] == class_label)][metric].to_numpy()
        res_mt_10 = df_res_10[(df_res_10['label_name'] == class_label)][metric].to_numpy()
        ttest_gt_res_10 = ttest_rel(gt_mt_10, res_mt_10)

        entry.append(np.round(ttest_gt_res_10.pvalue, 3))
    
    t.add_row(entry)

print(t)

+------------------------------------------------------+
|                    GT & ResNet50                     |
+------------------------------+-------+-------+-------+
|           Disease            |   AP  |   HR  |   RR  |
+------------------------------+-------+-------+-------+
|         lung opacity         |  0.0  |  0.0  | 0.034 |
|       pleural effusion       |  0.0  |  0.0  | 0.085 |
|         atelectasis          |  0.0  |  0.0  | 0.005 |
| enlarged cardiac silhouette  |  0.0  |  0.0  |  0.0  |
| pulmonary edema/hazy opacity |  0.0  |  0.0  |  0.0  |
|         pneumothorax         | 0.067 |  0.0  | 0.023 |
|        consolidation         | 0.035 | 0.286 | 0.054 |
| fluid overload/heart failure |  0.0  |  0.0  |  0.0  |
|          pneumonia           |  0.0  |  0.0  | 0.001 |
+------------------------------+-------+-------+-------+
